In [ ]:
import os

print(os.getcwd())


In [9]:
import asyncio
from TestClient import test_client

from sidecar import Sidecar
from ipyleaflet import *

icon_dict = { 
    test_client.UserStatus.NON_GROUP_MEMBER : 
    Icon(icon_url='http://getdrawings.com/free-icon/man-icon-64.png', icon_size=[38, 38]),
    test_client.UserStatus.GROUP_MEMBER : 
    Icon(icon_url='http://getdrawings.com/free-icon/man-icon-58.png', icon_size=[38, 38]),
    test_client.UserStatus.GROUP_LEADER : 
    Icon(icon_url='http://getdrawings.com/free-icon/leadership-icon-png-62.png', icon_size=[38, 38]) 
}

snu_position = (37.4556699,126.9533264)
num_clients = 15


m = Map(center=snu_position, zoom=15, basemap=basemaps.Esri.WorldStreetMap)

markers = []
marker_clusters = {}

for i in range(num_clients):
    markers.append(Marker(location=m.center,icon=icon_dict[test_client.UserStatus.NON_GROUP_MEMBER]))
    m += markers[i]
    
sc = Sidecar(title='Sidecar Output')
with sc:
    display(m)
    
def client_callback(clients):
    group_dict = {}
    for i in range(num_clients):
        markers[i].location = clients[i].position.tolist()
        markers[i].icon = icon_dict[clients[i].status]
        if clients[i].group_id is not None or clients[i].status is not test_client.UserStatus.NON_GROUP_MEMBER:
            group_dict.setdefault(clients[i].group_id, []).append(i)
    
    for marker_cluster in marker_clusters:
        m.remove_layer(marker_clusters)
    
    marker_clusters.clear()
    
    # change this to a circle
    for group_id, id_list in group_dict.items():
        group_markers = []
        for i in id_list:
            group_markers.append(markers[i])
        marker_clusters[group_id] = MarkerCluster(markers=group_markers)
        m.add_layer(marker_clusters[group_id])
    
    print(group_dict)
    
try:
    loop = asyncio.get_running_loop()
except RuntimeError:   
    loop = asyncio.get_event_loop()
    
task = asyncio.create_task(test_client.execute(loop, test_client.PositionUpdateTest, num_clients, 'http://localhost:8080/', client_callback))




In [10]:
print("Start task")

Start task


In [8]:
# You must cancel existing task here to re-run a script above.
# Resetting a event loop is not going to work properly 
# since it's shared with a IPython kernel
task.cancel()
sc.close()
